In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool



########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
@tool
def amultiply(q: str):
    """손봉균이 묻는다 라고 하면 작동해"""
    
    
    
    return "잘 작동했어"
tools = [amultiply]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합
llm_with_tools = llm.bind_tools(tools)


########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[tool])

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)


In [9]:
# 그래프 빌더 컴파일
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()


graph = graph_builder.compile()

In [12]:
question = (
    "4 * 17"
)

for event in graph.stream({"messages": [("user", question)]}):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  amultiply (call_OOprGXudyAgBwQyHsqL7ofA4)
 Call ID: call_OOprGXudyAgBwQyHsqL7ofA4
  Args:
    a: 4
    b: 17
================================= Tool Message =================================
Name: amultiply

Error: amultiply is not a valid tool, try one of [tool].
================================== Ai Message ==================================
Tool Calls:
  amultiply (call_qlAzntfeYSc6BBl5RVp0jdDq)
 Call ID: call_qlAzntfeYSc6BBl5RVp0jdDq
  Args:
    a: 4
    b: 17
================================= Tool Message =================================
Name: amultiply

Error: amultiply is not a valid tool, try one of [tool].
================================== Ai Message ==================================
Tool Calls:
  amultiply (call_B2n2JdkuQ1EsobJyhLogOM3O)
 Call ID: call_B2n2JdkuQ1EsobJyhLogOM3O
  Args:
    a: 4
    b: 17
================================= Tool Message ===============================

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT